In [1]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace,rand

In [2]:
def init_spark():
    spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
    sc = spark.sparkContext
    return spark, sc

# Exercise 1

# Exercise 2

First all the datasets are loaded. We converted the grundgesetzt to a txt-file. As well as created 9 other txt-files to test
our process on (Movie-scripts of:Harry Potter1-7 + Shrek1-2).

In [3]:
spark, sc = init_spark()

dataframes= []

dataframes.append(spark.read.text("data_sheet6/grundgesetz.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/hp1.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/hp2.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/hp3.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/hp4.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/hp5.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/hp6.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/hp7.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/shrek.txt",wholetext=True))
dataframes.append(spark.read.text("data_sheet6/shrek2.txt",wholetext=True))

22/12/02 16:20:50 WARN Utils: Your hostname, jakob-ThinkPad-E15-Gen-4 resolves to a loopback address: 127.0.1.1; using 192.168.111.224 instead (on interface wlp3s0)
22/12/02 16:20:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/02 16:20:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


 Now we need to do some processing on the text. We will replace the newline and tab characters with whitespaces.

In [4]:
for dataFrame in dataframes:
    # firstly we remove word seperations indicated by -\n
    dataFrame = dataFrame.withColumn("value",regexp_replace("value","-\n",""))
    #secondly we remove tabs and newlines
    dataFrame = dataFrame.withColumn("value",regexp_replace("value","\n"," "))
    dataFrame = dataFrame.withColumn("value",regexp_replace("value","\t",""))
    #lastly multiple whitespaces are removed and collapsed to a single one
    dataFrame = dataFrame.withColumn("value",regexp_replace("value","\\s{2,}"," "))

After cleaning up we can start to create the shingles. For that we first define a shingling function.

In [9]:
def shingling_k(text,k):
    tokens = list(text)
    shingle = [tokens[i:i+k] for i in range(len(tokens) - k + 1)]
    unique_shingles = []

    for shingleList in shingle:
        shingleText = "".join(str(i) for i in shingleList)
        unique_shingles.append(shingleText)
    return set(unique_shingles)

Now we can execute the singling functions for the different sets

In [17]:
name=['Grundgesetz','HP1','HP2','HP3','HP4','HP5','HP6','HP7','Shrek','Shrek2']
i=0
for dataFrame in dataframes:
    print(type(dataFrame))
    set_of_shingles_5 = dataFrame.rdd.map(lambda row: shingling_k((row[0]),5))
    set_of_shingles_9 = dataFrame.rdd.map(lambda row: shingling_k((row[0]),9))

    set_5 = set_of_shingles_5.take(1)
    set_9 = set_of_shingles_9.take(1)

    size_of_set_5 = len(set_5[0])
    size_of_set_9 = len(set_9[0])

    print("Created shingles for set: "+name[i])
    print("Amount of different 5 shingles: "+str(size_of_set_5))
    print("Amount of different 9 shingles: "+str(size_of_set_9))
    i += 1

<class 'pyspark.sql.dataframe.DataFrame'>
Created shingles for set: Grundgesetz
Amount of different 5 shingles: 26470
Amount of different 9 shingles: 83285
<class 'pyspark.sql.dataframe.DataFrame'>


Created shingles for set: HP1
Amount of different 5 shingles: 79295
Amount of different 9 shingles: 299736
<class 'pyspark.sql.dataframe.DataFrame'>


Created shingles for set: HP2
Amount of different 5 shingles: 82484
Amount of different 9 shingles: 338217
<class 'pyspark.sql.dataframe.DataFrame'>


Created shingles for set: HP3
Amount of different 5 shingles: 91605
Amount of different 9 shingles: 404934
<class 'pyspark.sql.dataframe.DataFrame'>


Created shingles for set: HP4
Amount of different 5 shingles: 118515
Amount of different 9 shingles: 645355
<class 'pyspark.sql.dataframe.DataFrame'>


Created shingles for set: HP5
Amount of different 5 shingles: 139262
Amount of different 9 shingles: 823049
<class 'pyspark.sql.dataframe.DataFrame'>


Created shingles for set: HP6
Amount of different 5 shingles: 113759
Amount of different 9 shingles: 594238
<class 'pyspark.sql.dataframe.DataFrame'>


Created shingles for set: HP7
Amount of different 5 shingles: 122966
Amount of different 9 shingles: 677162
<class 'pyspark.sql.dataframe.DataFrame'>
Created shingles for set: Shrek
Amount of different 5 shingles: 21090
Amount of different 9 shingles: 44410
<class 'pyspark.sql.dataframe.DataFrame'>
Created shingles for set: Shrek2
Amount of different 5 shingles: 20310
Amount of different 9 shingles: 39913


# Exercise 3

### a)

$$\begin{pmatrix}
0 & 1 & 0 & 1\\
0 & 1 & 0 & 0\\
1 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
0 & 0 & 1 & 1\\
1 & 0 & 0 & 0\\
\end{pmatrix}$$

#### Minhash for h_1(x)

$$\begin{pmatrix}
5 & 1 & 3 & 1
\end{pmatrix}$$

#### Minhash for h_2(x)

$$\begin{pmatrix}
2 & 2 & 2 & 2
\end{pmatrix}$$

#### Minhash for h_3(x)

$$\begin{pmatrix}
0 & 1 & 4 & 0
\end{pmatrix}$$


### b)

None of those functions provides a true permutation?

For h_1(x): S_2 and S_4 collide

For h_2(x): all S collide

For h_3(x): S_1 and S_4 collide

### c)

S_1 and S_2: Jaccard=0/4=0  Hashsim=1/3

S_1 and S_3: Jaccard=0/4=0  Hashsim=1/3

S_1 and S_4: Jaccard=1/4    Hashsim=2/3

S_2 and S_3: Jaccard=0/4=0  Hashsim=1/3

S_2 and S_4: Jaccard=1/4    Hashsim=2/3

S_3 and S_4: Jaccard=1/4    Hashsim=1/3